In [ ]:
import requests
import random
import pandas as pd
import time
import math
from copy import deepcopy
from statistics import mean

In [ ]:
def request() -> list:   
    res = requests.get('http://localhost:3000/api/ep1')
    return res.json()

In [ ]:
def data_treatment(json:list):
    for i in range(len(json)):
        for key in json[i]:
            if ('prod' in key):
                if (type(json[i][key])!=dict):
                    qnt = json[i][key]
                    if (type(qnt)==str):
                        qnt = float(qnt)                  
                    if (qnt<0):
                        qnt = 0
                    if (key in [product for product in json[i] if ('prod' in product and (int(product[5]) < 9 and len(product) < 7))]):
                        json[i][key] = {'qnt':[round(qnt)], 'price':None}
                    else:
                        json[i][key] = {'qnt':[qnt], 'price':None}
        try:
            json[i]['date'] = time.ctime(json[i]['date'])
        except:
            continue
    return json

In [ ]:
def generate_random_price(n_min=1.0, n_max=100.0) -> float:
    return round(random.uniform(n_min, n_max), 2)

In [ ]:
def logistic_function(v: float) -> float:
    return 0.5 + (1 / (1 + math.e ** -v))

In [ ]:
def calc_qnt_variation(last_week: list, current_week: list) -> float:
    return mean(current_week) - mean(last_week) / mean(last_week) if (mean(last_week) != 0) else 0

In [ ]:
def calc_new_price(product_db: dict, sales_week: dict) -> dict:
    for product in product_db:
        if (product_db[product]['qnt'] != None):
            if (product not in sales_week.keys()):
                sales_week[product] = [0]
            product_db[product]['next_week_price'] = logistic_function(calc_qnt_variation(product_db[product]['qnt'], sales_week[product])) * product_db[product]['price']
        product_db[product]['qnt'] = sales_week[product]               
    return product_db

In [ ]:
def product_pricing(json: list, product_db: dict) -> tuple:
    sales_week = {}

    for transaction in json:
        for key in transaction:
            if ('prod' in key):
                if (key not in product_db.keys()):
                    transaction[key]['price'] = generate_random_price()
                    sales_week[key] = deepcopy(transaction[key]['qnt'])
                    product_db[key] = deepcopy(transaction[key])
                    product_db[key]['qnt'] = None
                    product_db[key]['next_week_price'] = product_db[key]['price']              
                else:
                    transaction[key]['price'] = deepcopy(product_db[key]['next_week_price'])
                    product_db[key]['price'] = product_db[key]['next_week_price']
                    if (key not in sales_week.keys()):
                        sales_week[key] = deepcopy(transaction[key]['qnt'])
                    else:
                        sales_week[key] += deepcopy(transaction[key]['qnt'])

    product_db = calc_new_price(product_db, sales_week)

    return json, product_db

In [40]:

product_database = {}
weeks = 3
resumo = pd.DataFrame(columns = ['produto', 'soma', 'preço','receita','week'])


for week in range(weeks):
    json = data_treatment(request())
    json, product_database = product_pricing(json, product_database)

    json_db = pd.DataFrame(json)
    product_database_db = pd.DataFrame(product_database)

    print(f'\nSemana {week + 1}:\n')
    print('Json:')
    display(json_db)
    print('\nproduct_database:')
    display(product_database_db)
    semana = 'week' + str(week + 1)
    result_semanal =  resume(product_database_db, semana)
    resumo = resumo.merge(result_semanal, how = 'outer')
    



Semana 1:

Json:


,id,date,prod_1,prod_2,prod_5,prod_6,prod_8,prod_10,prod_15,prod_0,prod_3,prod_7,prod_4,prod_9,prod_11
0,laiulwr3wwhehaoxq0m,Wed Apr 1 23:12:52 2020,"{'qnt': [49], 'price': 16.03}","{'qnt': [40], 'price': 72.7}","{'qnt': [45], 'price': 33.32}","{'qnt': [36], 'price': 45.91}","{'qnt': [386], 'price': 85.46}","{'qnt': [0], 'price': 98.83}","{'qnt': [364.83534621744195], 'price': 75.45}",NaN,NaN,NaN,NaN,NaN,NaN
1,laiulwr31yyp72wh5ub,Wed Apr 1 21:06:05 2020,"{'qnt': [40], 'price': 16.03}","{'qnt': [22], 'price': 72.7}",NaN,"{'qnt': [18], 'price': 45.91}","{'qnt': [182], 'price': 85.46}",NaN,"{'qnt': [279.63321360895725], 'price': 75.45}","{'qnt': [8], 'price': 62.16}","{'qnt': [42], 'price': 54.9}","{'qnt': [24], 'price': 89.05}",NaN,NaN,NaN
2,laiulwr3ss5a1daadr,Thu Apr 2 07:57:21 2020,"{'qnt': [45], 'price': 16.03}",NaN,"{'qnt': [48], 'price': 33.32}",NaN,NaN,"{'qnt': [0], 'price': 98.83}","{'qnt': [444.2824972442067], 'price': 75.45}",NaN,"{'qnt': [44], 'price': 54.9}",NaN,"{'qnt': [42], 'price': 33.22}","{'qnt': [173.66930221094935], 'price': 11.36}",NaN
3,laiulwr31oga0zeeyvp,Thu Apr 2 01:22:38 2020,NaN,NaN,"{'qnt': [30], 'price': 33.32}",NaN,"{'qnt': [25], 'price': 85.46}",NaN,"{'qnt': [76.07728160553589], 'price': 75.45}","{'qnt': [1], 'price': 62.16}",NaN,NaN,"{'qnt': [7], 'price': 33.22}","{'qnt': [286.41857599079543], 'price': 11.36}",NaN
4,laiulwr4k5ayb5elh9q,Wed Apr 1 18:24:19 2020,NaN,NaN,NaN,NaN,"{'qnt': [257], 'price': 85.46}",NaN,"{'qnt': [0], 'price': 75.45}",NaN,NaN,NaN,"{'qnt': [22], 'price': 33.22}",NaN,NaN
5,laiulwr41ezetvt1vai,Thu Apr 2 06:05:11 2020,NaN,NaN,NaN,"{'qnt': [11], 'price': 45.91}","{'qnt': [319], 'price': 85.46}",NaN,"{'qnt': [290.2367947103429], 'price': 75.45}",NaN,NaN,"{'qnt': [15], 'price': 89.05}","{'qnt': [16], 'price': 33.22}","{'qnt': [0], 'price': 11.36}",NaN
6,laiulwr45cgfregm1wj,Wed Apr 1 18:02:22 2020,"{'qnt': [8], 'price': 16.03}","{'qnt': [31], 'price': 72.7}",NaN,NaN,"{'qnt': [11], 'price': 85.46}",NaN,"{'qnt': [332.7817535839848], 'price': 75.45}",NaN,"{'qnt': [13], 'price': 54.9}","{'qnt': [10], 'price': 89.05}","{'qnt': [14], 'price': 33.22}",NaN,"{'qnt': [417.65434141907133], 'price': 87.53}"
7,laiulwr449n3jvh7bfe,Thu Apr 2 01:08:48 2020,NaN,NaN,"{'qnt': [4], 'price': 33.32}","{'qnt': [30], 'price': 45.91}","{'qnt': [0], 'price': 85.46}","{'qnt': [0], 'price': 98.83}","{'qnt': [550.8472997484594], 'price': 75.45}","{'qnt': [12], 'price': 62.16}","{'qnt': [3], 'price': 54.9}",NaN,"{'qnt': [20], 'price': 33.22}","{'qnt': [2.1478965555901937], 'price': 11.36}",NaN
8,laiulwr4iejn4eafo8r,Thu Apr 2 00:20:44 2020,NaN,NaN,NaN,NaN,"{'qnt': [373], 'price': 85.46}","{'qnt': [274.31108577806344], 'price': 98.83}","{'qnt': [0], 'price': 75.45}",NaN,NaN,NaN,NaN,"{'qnt': [213.72857658410032], 'price': 11.36}","{'qnt': [30.779821266537795], 'price': 87.53}"
9,laiulwr498a7zy0si3,Wed Apr 1 18:58:46 2020,"{'qnt': [19], 'price': 16.03}","{'qnt': [14], 'price': 72.7}",NaN,NaN,"{'qnt': [318], 'price': 85.46}","{'qnt': [0], 'price': 98.83}","{'qnt': [199.74884972569615], 'price': 75.45}","{'qnt': [45], 'price': 62.16}","{'qnt': [44], 'price': 54.9}",NaN,"{'qnt': [4], 'price': 33.22}","{'qnt': [3.7647321930539306], 'price': 11.36}",NaN



product_database:


,prod_1,prod_2,prod_5,prod_6,prod_8,prod_10,prod_15,prod_0,prod_3,prod_7,prod_4,prod_9,prod_11
qnt,"[49, 40, 45, 8, 19, 4, 42, 11, 1, 43, 39, 22]","[40, 22, 31, 14, 30, 39, 24, 38, 28, 30, 11]","[45, 48, 30, 4, 45, 46, 31, 32, 9, 45, 44, 44,...","[36, 18, 11, 30, 12, 7, 11, 8, 22, 13, 20]","[386, 182, 25, 257, 319, 11, 0, 373, 318, 0, 0...","[0, 0, 0, 274.31108577806344, 0, 157.970798371...","[364.83534621744195, 279.63321360895725, 444.2...","[8, 1, 12, 45, 34, 14, 29, 16, 27, 33, 49, 1]","[42, 44, 13, 3, 44, 35, 31, 49, 48, 21, 2, 2, ...","[24, 15, 10, 37, 27, 45, 4, 27, 41, 30, 37, 49...","[42, 7, 22, 16, 14, 20, 4, 27, 47, 40, 0, 40, ...","[173.66930221094935, 286.41857599079543, 0, 2....","[417.65434141907133, 30.779821266537795, 192.3..."
price,16.03,72.7,33.32,45.91,85.46,98.83,75.45,62.16,54.9,89.05,33.22,11.36,87.53
next_week_price,16.03,72.7,33.32,45.91,85.46,98.83,75.45,62.16,54.9,89.05,33.22,11.36,87.53



Semana 2:

Json:


...

,prod_1,prod_2,prod_5,prod_6,prod_8,prod_10,prod_15,prod_0,prod_3,prod_7,prod_4,prod_9,prod_11
qnt,"[34, 2, 2, 35, 16, 17, 49]","[50, 30, 24, 41]","[24, 22, 40, 23, 47, 19]","[24, 24, 7, 26, 46, 23]","[785, 660, 477, 189, 293, 52, 192]","[369.6703954749935, 18.828233190169378, 687.34...","[328.6967098517389, 346.0486122440646, 205.574...","[34, 46, 35, 29, 32, 31]","[12, 37, 10, 7, 43, 31, 13]","[16, 29, 31, 42, 24]","[33, 44, 32, 7, 50, 45, 17, 19, 8]","[17.302134080774213, 562.5242769107597, 37.415...","[2.610835188235291, 150.24581870732527, 45.148..."
price,16.03,72.7,33.32,45.91,85.46,98.83,75.45,62.16,54.9,89.05,33.22,11.36,87.53
next_week_price,24.045,109.05,49.98,68.865,128.19,148.245,113.175,93.24,82.35,133.575,49.83,17.04,131.295



Semana 3:

Json:


,id,date,prod_1,prod_4,prod_5,prod_10,prod_15,prod_0,prod_2,prod_3,prod_8,prod_9,prod_6,prod_7,prod_11
0,laiulxb3z5z99heo3i,Thu Apr 16 05:20:14 2020,"{'qnt': [16], 'price': 24.044999989463236}","{'qnt': [1], 'price': 49.82999999995526}","{'qnt': [49], 'price': 49.9799999999805}","{'qnt': [1083.2943630635514], 'price': 148.245}","{'qnt': [982.1902244908545], 'price': 113.1750...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,laiulxb368484m3c5r9,Wed Apr 15 17:38:24 2020,"{'qnt': [2], 'price': 24.044999989463236}","{'qnt': [19], 'price': 49.82999999995526}",NaN,NaN,"{'qnt': [1117.3534129381223], 'price': 113.175...","{'qnt': [2], 'price': 93.23999999999981}","{'qnt': [13], 'price': 109.04999999999997}","{'qnt': [37], 'price': 82.3499999519791}","{'qnt': [708], 'price': 128.19}","{'qnt': [903.006780912428], 'price': 17.04}",NaN,NaN,NaN
2,laiulxb3on02wx10d9c,Wed Apr 15 20:20:30 2020,"{'qnt': [30], 'price': 24.044999989463236}",NaN,NaN,NaN,"{'qnt': [796.4884029272649], 'price': 113.1750...","{'qnt': [4], 'price': 93.23999999999981}","{'qnt': [21], 'price': 109.04999999999997}",NaN,"{'qnt': [26], 'price': 128.19}","{'qnt': [340.24612142476957], 'price': 17.04}","{'qnt': [32], 'price': 68.86499999826682}","{'qnt': [47], 'price': 133.5749999998878}",NaN
3,laiulxb3xbziqlin9xj,Thu Apr 16 03:14:58 2020,NaN,"{'qnt': [7], 'price': 49.82999999995526}","{'qnt': [34], 'price': 49.9799999999805}","{'qnt': [941.6180636463099], 'price': 148.245}","{'qnt': [459.12332565870327], 'price': 113.175...",NaN,NaN,"{'qnt': [32], 'price': 82.3499999519791}",NaN,"{'qnt': [24.150106735722645], 'price': 17.04}",NaN,"{'qnt': [37], 'price': 133.5749999998878}",NaN
4,laiulxb3mlx2iix3q4,Thu Apr 16 03:39:19 2020,NaN,NaN,NaN,"{'qnt': [863.1037473062052], 'price': 148.245}","{'qnt': [622.3897401574397], 'price': 113.1750...",NaN,NaN,"{'qnt': [17], 'price': 82.3499999519791}","{'qnt': [3], 'price': 128.19}","{'qnt': [370.3384041667053], 'price': 17.04}","{'qnt': [12], 'price': 68.86499999826682}",NaN,"{'qnt': [344.65521153672626], 'price': 131.295..."
5,laiulxb323ewmd4b1gc,Wed Apr 15 17:38:28 2020,"{'qnt': [20], 'price': 24.044999989463236}","{'qnt': [34], 'price': 49.82999999995526}","{'qnt': [9], 'price': 49.9799999999805}",NaN,"{'qnt': [1171.9342138731624], 'price': 113.175...","{'qnt': [44], 'price': 93.23999999999981}",NaN,NaN,NaN,NaN,"{'qnt': [44], 'price': 68.86499999826682}",NaN,NaN
6,laiulxb3wqwnc2zdwy,Thu Apr 16 08:33:10 2020,NaN,"{'qnt': [15], 'price': 49.82999999995526}",NaN,"{'qnt': [976.6939602373515], 'price': 148.245}","{'qnt': [895.4381651325161], 'price': 113.1750...",NaN,NaN,NaN,"{'qnt': [882], 'price': 128.19}",NaN,NaN,NaN,NaN
7,laiulxb3z7cs5oqh31,Thu Apr 16 01:46:46 2020,NaN,NaN,"{'qnt': [12], 'price': 49.9799999999805}",NaN,"{'qnt': [1078.1996548794255], 'price': 113.175...","{'qnt': [18], 'price': 93.23999999999981}",NaN,NaN,"{'qnt': [40], 'price': 128.19}","{'qnt': [7.391701901282255], 'price': 17.04}","{'qnt': [10], 'price': 68.86499999826682}",NaN,"{'qnt': [971.1355449824566], 'price': 131.2950..."
8,laiulxb37m3hvdblbn,Wed Apr 15 21:31:26 2020,NaN,NaN,"{'qnt': [45], 'price': 49.9799999999805}","{'qnt': [487.6158113611716], 'price': 148.245}","{'qnt': [527.1256883718837], 'price': 113.1750...","{'qnt': [46], 'price': 93.23999999999981}",NaN,NaN,"{'qnt': [657], 'price': 128.19}","{'qnt': [656.7430751830033], 'price': 17.04}","{'qnt': [30], 'price': 68.86499999826682}","{'qnt': [3], 'price': 133.5749999998878}","{'qnt': [687.9107948691383], 'price': 131.2950..."
9,laiulxb3ncjpvce2z7,Wed Apr 15 21:44:56 2020,"{'qnt': [44], 'price': 24.044999989463236}","{'qnt': [42], 'price': 49.82999999995526}",NaN,NaN,"{'qnt': [1017.7705169492648], 'price': 113.175...","{'qnt': [36], 'price': 93.23999999999981}",NaN,"{'qnt': [12], 'price': 82.3499999519791}","{'qnt': [394], 'price': 128.19}","{'qnt': [393.55433670480056], 'price': 17.04}",NaN,"{'qnt': [4], 'price': 133.5749999998878}","{'qnt': [885.2878834495539], 'price': 131.2950..."



product_database:


,prod_1,prod_2,prod_5,prod_6,prod_8,prod_10,prod_15,prod_0,prod_3,prod_7,prod_4,prod_9,prod_11
qnt,"[16, 2, 30, 20, 44, 24, 6, 10]","[13, 21, 11, 42, 19]","[49, 34, 9, 12, 45, 33, 4, 46, 13]","[32, 12, 44, 10, 30, 33, 45]","[708, 26, 3, 882, 40, 657, 394, 658, 803, 539,...","[1083.2943630635514, 941.6180636463099, 863.10...","[982.1902244908545, 1117.3534129381223, 796.48...","[2, 4, 44, 18, 46, 36, 43]","[37, 32, 17, 12, 38, 33, 13]","[47, 37, 3, 4, 8, 26, 18, 2]","[1, 19, 7, 34, 15, 42, 14, 41, 2]","[903.006780912428, 340.24612142476957, 24.1501...","[344.65521153672626, 971.1355449824566, 687.91..."
price,24.045,109.05,49.98,68.865,128.19,148.245,113.175,93.24,82.35,133.575,49.83,17.04,131.295
next_week_price,36.0675,163.575,74.97,103.2975,192.285,222.3675,169.7625,139.86,123.525,200.362495,74.745,25.56,196.9425


In [30]:

def resume (produtos, semana):

   df = produtos.copy()
   somas = []
   prices = []
   colunas = list(df.columns)

   for i in range(df.shape[1]):
      somas.append(sum(df.iloc[0,i]))
      prices.append(df.iloc[1,i])
      
   lista_de_tuplas = list(zip(colunas, somas, prices))
   new_df = pd.DataFrame(lista_de_tuplas, columns = ['produto', 'soma', 'preço'])
   new_df['receita'] = new_df['soma'] * new_df['preço']
   
   new_df['week'] = semana
   
   return new_df
# este código já me dá a receita por semana, mas não por venda

In [41]:
resumo

,produto,soma,preço,receita,week
0,prod_1,323.000000,16.030,5.177690e+03,week1
1,prod_2,307.000000,72.700,2.231890e+04,week1
2,prod_5,494.000000,33.320,1.646008e+04,week1
3,prod_6,188.000000,45.910,8.631080e+03,week1
4,prod_8,3338.000000,85.460,2.852655e+05,week1
5,prod_10,1054.971436,98.830,1.042628e+05,week1
6,prod_15,4657.876846,75.450,3.514368e+05,week1
7,prod_0,269.000000,62.160,1.672104e+04,week1
8,prod_3,427.000000,54.900,2.344230e+04,week1
9,prod_7,375.000000,89.050,3.339375e+04,week1
